<a href="https://colab.research.google.com/github/parisazeynaly/heart_disease-DeepLearning/blob/main/heart_disease_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>